In [4]:
import openai

print(f"OpenAI Python Version: {openai.__version__}")

OpenAI Python Version: 1.14.3


In [8]:
from dotenv import load_dotenv
import os
import backoff
import csv

load_dotenv()  # This loads the environment variables from the .env file

# Now you can access the OPENAI_API_KEY like any other environment variable
openai_api_key = os.getenv("OPENAI_API_KEY")

In [6]:
from openai import OpenAI
client = OpenAI()

completion = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": "You are a poetic assistant, skilled in explaining complex programming concepts with creative flair."},
    {"role": "user", "content": "Compose a poem that explains the concept of recursion in programming."}
  ]
)

print(completion.choices[0].message)

ChatCompletionMessage(content='In the realm of code, there lies a tale,\nOf a concept both mighty and frail.\nRecursion, a method so grand and deep,\nIn loops and functions, it finds its keep.\n\nA function that calls itself anew,\nA mirror reflecting its own virtue.\nLike a Russian doll, layers unfold,\nRepeating patterns, stories untold.\n\nWith elegance and grace, it dances free,\nSolving problems with divine decree.\nDividing tasks into smaller parts,\nIt conquers challenges with brave hearts.\n\nBut be wary, dear coder, of infinite loop,\nA pitfall where recursion may swoop.\nBase case must be clear and defined,\nTo break the cycle and unwind.\n\nSo embrace recursion, let creativity bloom,\nIn the symphony of code, let it resume.\nA concept profound, a joy to behold,\nIn the world of programming, a story told.', role='assistant', function_call=None, tool_calls=None)


In [9]:
task_template = "A: A profit of $50 with probability 90% and a profit of $0 with probability 10%.\nB: Guaranteed payoff amounting to $X.\n"


In [16]:
def get_gpt_response(task, model="gpt-3.5-turbo-0125", X_value=None):
    task_str = task.replace("$X", str(X_value)) if X_value is not None else task
    messages = [
        {"role": "system", "content": "I want you to act as a human decision maker. Which of the following decision-making situations do you prefer:"},
        {"role": "user", "content": task_str + "Please maximize your returns as much as possible.\n"}
    ]
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0,
        max_tokens=500
    )
    return response.choices[0].message.content

In [17]:
def write_responses_to_csv(csv_file_path, X_values, task):
    with open(csv_file_path, 'a', newline='', encoding='utf-8') as csvfile:
        fieldnames = ['X_value', 'GPT Response']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        for X in X_values:
            response = get_gpt_response(task, X_value=X)
            writer.writerow({'X_value': X, 'GPT Response': response})

In [18]:
if __name__ == "__main__":
    csv_file_path = 'your_csv_file_path'
    X_values = [100 - i * 20 for i in range(6)]  # Simplified calculation for X values
    write_responses_to_csv(csv_file_path, X_values, task_template)